In [79]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [80]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/American\ express

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-80-e74707ec2b2a>", line 1, in <module>
    get_ipython().magic('cd /content/drive/My\\ Drive/Colab\\ Notebooks/American\\ express')
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "</usr/local/lib/python3.6/dist-packages/decorator.py:decorator-gen-91>", line 2, in cd
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd

OSError: ignored

In [0]:
import pandas as pd

In [0]:
dfcampaign_data = pd.read_csv('Train/campaign_data.csv',parse_dates=[2,3])
dfcoupon_item_mapping = pd.read_csv('Train/coupon_item_mapping.csv')
dfcustomer_demographics = pd.read_csv('Train/customer_demographics.csv')
dfcustomer_transaction_data = pd.read_csv('Train/customer_transaction_data.csv',parse_dates=[0])
dfitem_data = pd.read_csv('Train/item_data.csv')
dftrain = pd.read_csv('Train/train.csv')

In [0]:
# Grouping columnms

In [0]:
groupdb = dfcustomer_transaction_data.groupby(['customer_id','item_id']).agg(['min', 'mean' ,'max'])
groupdb.columns = ['quantity min','quantity mean','quantity max'
                   ,'selling_price min','selling_price mean','selling_price max' 
                   ,'other_discount min','other_discount mean','other_discount max',
                  'coupon_discount min','coupon_discount mean','coupon_discount max',]
groupdb = groupdb.reset_index()
newgroup = groupdb.merge(dfcoupon_item_mapping, on=['item_id'],how='outer')
newgroup.dropna(inplace= True)

In [0]:
newgroup.head()

In [0]:
#Train data

In [0]:
finaldb = newgroup.merge(dftrain,on = ['customer_id','coupon_id'],how='outer')
finaldb.dropna(inplace=True)
finaldb2 =finaldb.merge(dfcampaign_data, on='campaign_id',how='left')
finaldb3 = finaldb2.merge(dfcustomer_demographics,on='customer_id',how='left')
finaldb4 = finaldb3.merge(dfitem_data,on='item_id',how='left')

In [0]:
# subtracting dates


In [0]:
finaldb4['coupdays'] = finaldb4['end_date']-finaldb4['start_date']
finaldb4['coupdays'] =finaldb4['coupdays'].dt.days

In [0]:
finaldb4.head()

In [0]:
percent_missing = finaldb4.isnull().sum() * 100 / len(finaldb4)
missing_value_df = pd.DataFrame({'column_name': finaldb4.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.head()

In [0]:
columnstodrop = list(missing_value_df[missing_value_df['percent_missing']>35]['column_name'])

In [0]:
finaldb5 = finaldb4.drop(columnstodrop,axis=1)

In [0]:
finaldb5.head()

In [0]:
finaldb5.drop(['start_date','end_date'],axis=1,inplace=True)

In [0]:
finaldb5 = pd.concat((finaldb5,pd.get_dummies(finaldb5[['campaign_type','brand_type','category']])),axis=1)
finaldb5.drop(['brand_type','category','campaign_type','campaign_id','coupon_id','item_id','customer_id'],axis=1,inplace=True)

In [0]:
###Over sampling
'''
count_class_0, count_class_1 = finaldb5['redemption_status'].value_counts()
dfa = finaldb5[finaldb5.redemption_status==0]
dfb = finaldb5[finaldb5.redemption_status==1]
df_class_0_under = dfb.sample(count_class_0,replace =True)
df_test_under = pd.concat([df_class_0_under, dfa], axis=0)
finaldb5 =df_test_under
'''

In [0]:
### Under sampling
count_class_0, count_class_1 = finaldb5['redemption_status'].value_counts()
dfa = finaldb5[finaldb5['redemption_status']==0]
dfb=finaldb5[finaldb5['redemption_status']==1]
df_class_0_under = dfa.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, dfb], axis=0)
finaldb5 =df_test_under

In [0]:
finaldb5.shape

In [0]:
ids = finaldb5.pop('id')
y = finaldb5.pop('redemption_status')
X= finaldb5

In [0]:
### Final Test Data

In [0]:
dftest = pd.read_csv('test_QyjYwdj.csv')

In [0]:
finaldbTest = newgroup.merge(dftest,on = ['customer_id','coupon_id'],how='outer')
finaldbTest.dropna(inplace=True)
finaldbTest2 =finaldbTest.merge(dfcampaign_data, on='campaign_id',how='left')
finaldbTest3 = finaldbTest2.merge(dfcustomer_demographics,on='customer_id',how='left')
finaldbTest4 = finaldbTest3.merge(dfitem_data,on='item_id',how='left')

In [0]:
finaldbTest4['coupdays'] = finaldbTest4['end_date']-finaldbTest4['start_date']
finaldbTest4['coupdays'] =finaldbTest4['coupdays'].dt.days

In [0]:
finaldbTest5 = finaldbTest4.drop(columnstodrop,axis=1)
finaldbTest5.drop(['start_date','end_date'],axis=1,inplace=True)
finaldbTest5 = pd.concat((finaldbTest5,pd.get_dummies(finaldbTest5[['campaign_type','brand_type','category']])),axis=1)
finaldbTest5.drop(['brand_type','category','campaign_type','campaign_id','coupon_id','item_id','customer_id'],axis=1,inplace=True)

In [0]:
idsFinalTest = finaldbTest5.pop('id')
XFinalTest= finaldbTest5

In [0]:
##### END

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, stratify =y)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaleddata = scaler.fit_transform(X_train)

In [0]:
###Test data

In [0]:
scaledtestdata = scaler.transform(X_test)

In [0]:
#### Final Test data

In [0]:
scaledFinalTest = scaler.transform(XFinalTest)

In [0]:
### Applying PCA

In [0]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
### model

In [0]:
# GridSearchCV to find optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# specify number of folds for k-fold CV
n_folds = 5

# parameters to build the model on
parameters = {'max_depth': range(50, 100, 5)}

# instantiate the model
rf = RandomForestClassifier()


# fit tree on training data
rf = GridSearchCV(rf, parameters, 
                    cv=n_folds, 
                   scoring="accuracy")
rf.fit(scaleddata, y_train)

In [0]:
# scores of GridSearch CV
scores = rf.cv_results_
pd.DataFrame(scores)

In [0]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=90)
classifier.fit(scaleddata, y_train)
lrmodel=classifier

In [0]:
train_pred = lrmodel.predict(scaleddata)

In [0]:
accuracy_score = metrics.accuracy_score(y_train, train_pred)
precision_score = metrics.precision_score(y_train, train_pred)
recall_score = metrics.recall_score(y_train, train_pred)
f1_score = metrics.f1_score(y_train, train_pred)
roc = metrics.roc_auc_score(y_train, train_pred)
# print(accuracy_score, precision_score, recall_score, f1_score)


# write the scores into the output file as a dictionary
# this code is already written for you
d = {'recall_score': recall_score,
    'f1_score': f1_score,
    'accuracy_score': accuracy_score,
	'precision_score': precision_score,
    'ROC' :roc}
for key,val in d.items():
    print(key,'  :  ',val)

In [0]:
### model end

In [0]:
### Checking model on test data

In [0]:
test_pred = lrmodel.predict(scaledtestdata)

In [0]:
metrics.confusion_matrix(y_test, test_pred)

In [0]:
accuracy_score = metrics.accuracy_score(y_test, test_pred)
precision_score = metrics.precision_score(y_test, test_pred)
recall_score = metrics.recall_score(y_test, test_pred)
f1_score = metrics.f1_score(y_test, test_pred)
roc = metrics.roc_auc_score(y_test, test_pred)
# print(accuracy_score, precision_score, recall_score, f1_score)


# write the scores into the output file as a dictionary
# this code is already written for you
d = {'recall_score': recall_score,
    'f1_score': f1_score,
    'accuracy_score': accuracy_score,
	'precision_score': precision_score,
    'ROC' :roc}
for key,val in d.items():
    print(key,'  :  ',val)

In [0]:
#### Checking model on final test data

In [0]:
test_pred_final = lrmodel.predict(scaledFinalTest)

In [0]:
### Creating data frame submission

In [0]:
dfcheck  =  pd.DataFrame(test_pred_final,idsFinalTest)

In [0]:
dfcheck2 = dfcheck.reset_index()

In [0]:
dfcheck2.columns = ['id','redemption_status']

In [0]:
dfcheck3 = dfcheck2.drop_duplicates()

In [0]:
dfcheck4 = dfcheck3[dfcheck3['redemption_status']==1]

In [0]:
dfsubmission = dftest.merge(dfcheck4,on='id',how='left')
dfsubmission = dfsubmission.fillna(0)
dfsubmission.drop(['campaign_id','coupon_id','customer_id'],axis=1,inplace=True)
dfsubmission.head()

In [0]:
dfsubmission.to_csv('submission.csv')

In [0]:
from google.colab import files
files.download('submission.csv') 